In [1]:
#import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"  

%matplotlib inline

from pathlib import Path
import sys
sys.path.append("../../")

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

from tqdm.auto import tqdm

import captchanet

data_dir = Path('/home/hadim/.data/Neural_Network/captchanet')

dataset_dir = data_dir / 'dataset_v6'
train_data_dir = dataset_dir / 'training'
val_data_dir = dataset_dir / 'validation'

model_dir = data_dir / 'model'
model_name = 'v6'
model_path = model_dir / model_name

tokenizer_path = model_path / "tokenizer.json"

# Get tokenizer
with open(tokenizer_path) as f:
  #tokenizer = tf.keras.preprocessing.text.tokenizer_from_json(f.read())
  from keras_preprocessing import text
  tokenizer = text.tokenizer_from_json(f.read())
  
model = tf.keras.models.load_model(str(model_path / 'model'))

In [2]:
# Define a model that do image preprocessing before inference.
class CaptchaSolverInferenceModel(tf.keras.Model):
  
  def __init__(self, captchanet_model, **kwargs):
    super().__init__(**kwargs)
    self.captchanet_model = captchanet_model

  @tf.function(input_signature=[tf.TensorSpec(shape=(None, None, None, 3), dtype=tf.uint8)])
  def solve(self, images):
    images = tf.cast(images, tf.float32)
    # Normalize images
    images = tf.image.per_image_standardization(images)
    images = tf.image.resize(images, (224, 224))
    
    # Inference
    labels = self.captchanet_model(images)
    labels = tf.argmax(labels, axis=2)
    
    # Freeze the tokenizer
    n_letter = max(tokenizer.index_word.keys()) + 1
    index = [''] * n_letter
    for k, v in tokenizer.index_word.items():
      index[k] = v
    index = tf.constant(index, dtype=tf.string)
        
    # Decode labels
    decoder = lambda x: tf.gather_nd(index, tf.expand_dims(x, axis=-1))
    words = tf.map_fn(decoder, labels, dtype=tf.string)
    joiner = lambda x: tf.strings.reduce_join(x, axis=0)
    words = tf.map_fn(joiner, words, dtype=tf.string)

    return words

  
inference_model = CaptchaSolverInferenceModel(model)
inference_model_path = model_path / 'inference_model'
tf.saved_model.save(inference_model, str(inference_model_path))

W0714 00:36:05.113139 139779263436608 saved_model.py:758] Skipping full serialization of Keras model <__main__.CaptchaSolverInferenceModel object at 0x7f1b2615f668>, because its inputs are not defined.
W0714 00:36:05.645812 139779263436608 deprecation.py:323] From /home/hadim/conda/envs/captchanet/lib/python3.7/site-packages/tensorflow/python/ops/image_ops_impl.py:1511: div (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
W0714 00:36:06.461832 139779263436608 saved_model.py:765] Skipping full serialization of Keras layer <tensorflow.python.keras.saving.saved_model.InputLayer object at 0x7f1efb146588>, because it is not built.
W0714 00:36:06.463783 139779263436608 saved_model.py:765] Skipping full serialization of Keras layer <tensorflow.python.keras.saving.saved_model.KerasLayer object at 0x7f1efb146668>, because it is not built.
W0714 00:36:06.466694 13977926343660

In [3]:
# Pack and zip the model directory
import shutil
archive_path = model_dir / model_path.stem
shutil.rmtree(model_path / '.ipynb_checkpoints', ignore_errors=True)
shutil.make_archive(archive_path, 'zip', root_dir=model_path)

'/home/hadim/.data/Neural_Network/captchanet/model/v6.zip'

In [4]:
# Test the exported model

inference_model = tf.saved_model.load(str(model_path / 'inference_model'))

model_image_size = (224, 224)
n  = 8
fnames = [str(p) for p in val_data_dir.glob("*.tfrecord")]
dataset = tf.data.TFRecordDataset(fnames)
dataset = dataset.map(map_func=captchanet.decode_data(tokenizer, image_size=model_image_size, max_len_word=10, input_as_dict=True))
dataset = dataset.shuffle(1024)
dataset = dataset.batch(n)
data = [d for d in dataset.take(1)][0]

images = data['original_image']
labels = data['label']
labels = tf.argmax(labels, axis=2)

# Solve captcha
predicted_words = inference_model.solve(images)
predicted_words = predicted_words.numpy()

# Plot
original_images = data['original_image'].numpy()
words = data['word'].numpy()
words = [w.decode('utf-8').replace('0', '') for w in words]

size = 2
ncols = 2
nrows = n // ncols
ratio = original_images.shape[2] / original_images.shape[1]
figsize = (ncols * size * ratio, size * nrows)
fig, axs = plt.subplots(nrows=nrows, ncols=ncols, figsize=figsize, constrained_layout=True)
axs = axs.flatten()

for image, word, predicted_word, ax in zip(images, words, predicted_words, axs):
  ax.imshow(image)
  
  predicted_word = predicted_word.decode()
  mark = 'OK' if predicted_word == word else 'WRONG'
  text = f'True: {word} ({len(word)})'
  text += f' - Predicted: {predicted_word} ({len(predicted_word)})'
  text += f" - {mark}"
  ax.set_title(text, fontsize=14)

InvalidArgumentError: 2 root error(s) found.
  (0) Invalid argument:  2 root error(s) found.
  (0) Invalid argument: During Variant Host->Device Copy: non-DMA-copy attempted of tensor type: string
  (1) Invalid argument: During Variant Host->Device Copy: non-DMA-copy attempted of tensor type: string
0 successful operations.
0 derived errors ignored.
	 [[{{node StatefulPartitionedCall/map/while/body/_567/TensorArrayV2Write/TensorListSetItem/_58}}]]
  (1) Invalid argument:  2 root error(s) found.
  (0) Invalid argument: During Variant Host->Device Copy: non-DMA-copy attempted of tensor type: string
  (1) Invalid argument: During Variant Host->Device Copy: non-DMA-copy attempted of tensor type: string
0 successful operations.
0 derived errors ignored.
	 [[{{node StatefulPartitionedCall/map/while/body/_567/TensorArrayV2Write/TensorListSetItem/_58}}]]
	 [[Func/StatefulPartitionedCall/map_1/while/body/_608/input/_967/_72]]
0 successful operations.
0 derived errors ignored. [Op:__inference_re